In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from urllib.parse import urlparse, parse_qs, parse_qsl
import time
from datetime import datetime, timedelta
import re
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [2]:
service = Service()
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=service, options=options)

In [3]:
site_url = "https://www.teamblind.com/kr/topics/부동산"

In [ ]:
driver.get(site_url)
driver.implicitly_wait(3)
time.sleep(2)  # 페이지 로딩 대기

In [4]:
# Get Link on div
sub_link = driver.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').get_attribute('href')
sub_link

'https://www.teamblind.com/kr/post/%EB%8B%A4%EB%A5%B8%EA%B1%B4-%EB%AA%B0%EB%9D%BC%EB%8F%84-%EB%8F%88-%EB%A7%8E%EC%9D%80-%EC%82%AC%EB%9E%8C%EC%9D%B4-%EB%A7%8E%EB%8B%A4%EB%8A%94-%EB%A7%90%EC%9D%80-%EA%B0%9C%EA%B5%AC%EB%9D%BC%EC%95%BC%E3%85%8E%E3%85%8E-26nihsck'

In [ ]:
# 게시글 진입
driver.get(sub_link)

In [ ]:
# 게시글 + 댓글 크롤링 start
# 제목 (<h2>)
driver.find_element(By.CLASS_NAME, "article-view-head").find_element(By.CSS_SELECTOR, "h2").text

'다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ'

In [38]:
# 게시글
driver.find_element(By.CLASS_NAME, "article-view-contents").find_element(By.ID, "contentArea").text

'내가 담당했던 업무 (지금은 아님)가\n\nKb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩\n\n당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료\n\n그거보면서 느낀게\n\n돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ\n\n대부분이 빚더미 위에 올라서 돈많은 척하는거지\n\n솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음\n\n지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는게 웃음벨\n\n통계에 금융자산 10억이 우리나라에 0.8%도 안되는게 레알인데 저기엔 채권이나 펀드 주식 등 모든 금융자산이 포함되어있어서 당장 현금화시킬수있는 돈 기준으로 하면 진짜 돈많은 사람은 거의 없다\n\n한마디로 집 빼면 대부분 빚더미에 앉아있는 위태위태한 사람들이 거의 다더라..'

In [ ]:
# 작성회사
driver.find_element(By.CLASS_NAME, "name").find_element(By.CLASS_NAME, "point").text

'SC제일은행'

In [25]:
# 작성ID
driver.find_element(By.CLASS_NAME, "name").text.split('\n')[-1]

'· l********  '

In [4]:
# 작성일(절대날짜 추출)
def convert_date(relative_text):
    """
    상대적인 시간 표현을 절대 날짜로 변환하는 함수
    
    Args:
        relative_text (str): '방금', '2분', '1시간', '1일', '1달' 등의 텍스트
    
    Returns:
        str: '%Y-%m-%d' 형태의 절대 날짜
    """
    # 현재 날짜 및 시간
    now = datetime.now()
    # 공백 제거 및 소문자 변환
    text = relative_text.strip()
    # '방금'인 경우 오늘 날짜 반환
    if text == '방금':
        return now.strftime('%Y-%m-%d')
    
    # 숫자와 단위 추출을 위한 정규표현식
    pattern = r'(\d+)(분|시간|일|달|주)'
    match = re.search(pattern, text)
    
    if not match:
        # 패턴이 매치되지 않으면 오늘 날짜 반환
        return now.strftime('%Y-%m-%d')
    
    number = int(match.group(1))
    unit = match.group(2)
    
    # 단위별로 날짜 계산
    if unit == '분':
        target_date = now - timedelta(minutes=number)
    elif unit == '시간':
        target_date = now - timedelta(hours=number)
    elif unit == '일':
        target_date = now - timedelta(days=number)
    elif unit == '주':
        target_date = now - timedelta(weeks=number)
    elif unit == '달':
        # 달의 경우 대략 30일로 계산 (정확하지 않을 수 있음)
        target_date = now - timedelta(days=number * 30)
    else:
        target_date = now
    
    return target_date.strftime('%Y-%m-%d')

In [32]:
# 작성일(현재시점 며칠 전)
write_date_before=driver.find_element(By.CLASS_NAME, "date").text.split('\n')[-1]
write_date_before

'4일'

In [ ]:
# 작성일(절대시간 기준)
convert_date(write_date_before)

'2025-06-28'

In [35]:
# 좋아요
driver.find_element(By.CLASS_NAME, "like").text.split('\n')[-1]

'201'

In [37]:
# 댓글수
driver.find_element(By.CLASS_NAME, "cmt").text.split('\n')[-1]

'145'

In [ ]:
# 댓글 세부내용 추출
# 댓글이 더 있는경우 모두 펼친다
# 이 기능은 로그인 되어있는 경우만 가능
driver.find_element(By.CLASS_NAME, "btn-reply").click()

In [ ]:
# Blind 인증 요구 시 다음 Element 가 생긴다
# 해당 Element가 존재한다면 로그인 불가로 간주하고 현재 보여지는 댓글만 가져오고 끝낸다
# 해당 Element가 없다면, try-catch 로 계속 진행하도록 수행(로그인 된 것으로 간주)
driver.find_element(By.XPATH, "/html/body/div/div/div/main/div[3]/div[1]/div")

<selenium.webdriver.remote.webelement.WebElement (session="8483d6fa5b242ce85251fb8bae404044", element="f.6FE7B10A4ABC9024921797A9D4DD568D.d.B505AED63D3E7AA3F6906F3382FF69E0.e.11677")>

In [79]:
comment_element_list=driver.find_elements(By.CLASS_NAME, "comment_area")
comment_element_list[2].text

'한국전력공사\n· i*********\n또또 주변도르\n작성일\n3일\n좋아요수\n14'

In [102]:
print(comment_element_list[7].text.split('·')[0]) # 리플 작성회사
print(comment_element_list[7].text.split('·')[1].split('\n')[0]) # 리플 작성ID
print(comment_element_list[7].text.split('·')[1].split(comment_element_list[7].text.split('·')[1].split('\n')[0])[1].split('작성일')[0]) # 리플 내용

한양대학교교직원

 i*******

와 중소기업연구원은
ㄹㅇ 멍청한게신념도 강해서
때려죽여도 남의말은 안듣네

너무추하다ㅋㅋ



In [105]:
def get_comment_txt(comment) :
    comment_map = {}
    comment_map['firm']=comment.text.split('·')[0] # 리플 작성회사
    comment_map['id']=comment.text.split('·')[1].split('\n')[0]  # 리플 작성ID
    # comment_map['contents']=comment.text.split('·')[1].split('\n')[1]  # 리플 내용
    comment_map['contents']=comment.text.split('·')[1].split(comment.text.split('·')[1].split('\n')[0])[1].split('작성일')[0] # 리플 내용
    
    return comment_map

In [119]:
comment_list = list(map(lambda x : get_comment_txt(x), comment_element_list))
comment_list

[{'firm': '변호사 ', 'id': ' P*****', 'contents': '\n30살 연봉 1억이면 상위 몇퍼야\n'},
 {'firm': '새회사 ',
  'id': ' 세********',
  'contents': '\n나 만 28인데 부채제외 주식 부동산 차량 다 현금화하면 7억은 됨 근데 내 주변에 친구들은 다 더 부자야\n생각보다 부자 많어 ㅠㅡㅠ\n'},
 {'firm': '한국전력공사\n', 'id': ' i*********', 'contents': '\n또또 주변도르\n'},
 {'firm': '새회사 ',
  'id': ' I********',
  'contents': '\n우리나라에 순자산 32억 이상 있는 가구는 20만이 넘어\n\n“상위 1%“인 20만 가구는 서울의 한 구 전체, 혹은 중소도시 하나를 통째로 구성할 정도로 엄청나게 큰 집단입니다.\n단순히 ‘극소수’가 아닌, 꽤 두터운 규모의 인원이라는 걸 알 수 있죠.\n\n참고로 금융자산 10억 이상은 46만명임\n46만이면 성남시 분당구 인구수랑 같음\n\n이렇게 많은 사람들이 5000만명 대비는 작으니까 우리나라 부자 ㅈ도없엌ㅋㅋㅋㅋ 라고할수 있을까?\n\n손바닥으로 태양을 가리고 어둡다고 하는거지\n\n그리고 부자들을 모여살아 끼리끼리\n\n본인주변에 부자가 없다면\n왜일까 생각해 보자\n'},
 {'firm': 'SC제일은행\n',
  'id': ' l********',
  'contents': '\n작성자\nㅋㅋㅋ 내가 그 순자산 32억이상 있는 가구중 한명인데. 나도 부동산에 묶여있어서 사실 현금이 별로없어. 그리고 현금화? 그게 말이쉽지 부동산 파는건 쉬운일이 아니다. 양도세도 그렇고\n\n대다수 32억이상 있는 가구들도 나와 사정이 별반 다르지않을거야 그러니 금융자산 있는 사람들이 우리나라엔 별로없는것임.\n'},
 {'firm': '신영증권\n',
  'id': ' 신****',
  'contents': '\nㅋㅋㅋ 은행이 자료를가지고 말해도\n"주변도르" 

In [6]:
# 게시물 이동 후 모든 컨텐츠를 가져오는 함수 definition
def get_article(link) : 
    try :
        driver.get(link)
        time.sleep(2)
        article = {}
        # 제목 (<h2>)
        title=driver.find_element(By.CLASS_NAME, "article-view-head").find_element(By.CSS_SELECTOR, "h2").text
        # 게시글
        body=driver.find_element(By.CLASS_NAME, "article-view-contents").find_element(By.ID, "contentArea").text
        # 작성회사
        writer_firm=driver.find_element(By.CLASS_NAME, "name").find_element(By.CLASS_NAME, "point").text
        # 작성ID
        writer_id=driver.find_element(By.CLASS_NAME, "name").text.split('\n')[-1]

        # 작성일
        write_date_before=driver.find_element(By.CLASS_NAME, "date").text.split('\n')[-1]
        write_date=convert_date(write_date_before)

        # 좋아요
        like_cnt=driver.find_element(By.CLASS_NAME, "like").text.split('\n')[-1]

        # 댓글수
        comment_cnt=driver.find_element(By.CLASS_NAME, "cmt").text.split('\n')[-1]

        # 댓글 세부내용 추출
        # 댓글이 더 있는경우 모두 펼친다
        # 이 기능은 로그인 되어있는 경우만 가능
        while True :
            # Blind 인증 요구 시 다음 Element 가 생긴다
            # 해당 Element가 존재한다면 로그인 불가로 간주하고 현재 보여지는 댓글만 가져오고 끝낸다
            # 해당 Element가 없다면, try-catch 로 계속 진행하도록 수행(로그인 된 것으로 간주)
            try :
                try :
                    driver.find_element(By.CLASS_NAME, "btn-reply").click()
                except :
                    break
                
                if(driver.find_element(By.XPATH, "/html/body/div/div/div/main/div[3]/div[1]/div")) :
                    # driver.find_element(By.CLASS_NAME, "btn-cls").click()
                    driver.find_element(By.CSS_SELECTOR, "#wrap > div:nth-child(4) > div:nth-child(1) > div > div.v--modal-box.v--modal.v--size-s > div > div.ly-modal.ly-signin > div.top-area > button").click()
                    break
            except :
                continue
            
    
        comment_element_list=driver.find_elements(By.CLASS_NAME, "comment_area")
        comment_map = [get_comment_txt(x) for x in comment_element_list]
        
        # Make tuple
        article['title']=title
        article['body']=body
        article['firm']=writer_firm
        article['id']=writer_id
        article['like_cnt']=like_cnt
        article['comment_cnt']=comment_cnt
        article['comment_map']=comment_map
        
        print(f"[세부 글] {article}")
    
        driver.back()
    except :
        return None
 
    return article

In [141]:
get_article(sub_link)

{'title': '다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ',
 'body': '내가 담당했던 업무 (지금은 아님)가\n\nKb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩\n\n당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료\n\n그거보면서 느낀게\n\n돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ\n\n대부분이 빚더미 위에 올라서 돈많은 척하는거지\n\n솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음\n\n지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는게 웃음벨\n\n통계에 금융자산 10억이 우리나라에 0.8%도 안되는게 레알인데 저기엔 채권이나 펀드 주식 등 모든 금융자산이 포함되어있어서 당장 현금화시킬수있는 돈 기준으로 하면 진짜 돈많은 사람은 거의 없다\n\n한마디로 집 빼면 대부분 빚더미에 앉아있는 위태위태한 사람들이 거의 다더라..',
 'firm': 'SC제일은행',
 'id': '· l********  ',
 'like_cnt': '201',
 'comment_cnt': '145',
 'comment_map': [{'firm': '변호사 ',
   'id': ' P*****',
   'contents': '\n30살 연봉 1억이면 상위 몇퍼야\n'},
  {'firm': '새회사 ',
   'id': ' 세********',
   'contents': '\n나 만 28인데 부채제외 주식 부동산 차량 다 현금화하면 7억은 됨 근데 내 주변에 친구들은 다 더 부자야\n생각보다 부자 많어 ㅠㅡㅠ\n'},
  {'firm': '한국전력공사\n', 'id': ' i*********', 'contents': '\n또또 주변도르\n'},
  {'firm': '새회사 ',
   'id': ' I********',
   'contents': '\n우리나라에 순자산 32억

In [ ]:
# Get Pre-Text on div
driver.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').text

'전세가 병신짓인걸 모르는 사람이 많다는게 놀랍다'

In [146]:
# 최초 설정
a=driver.find_element(By.CLASS_NAME, "article-list-pre")
a.text

'NOW\n다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ\n내가 담당했던 업무 (지금은 아님)가 Kb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩 당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료 그거보면서 느낀게 돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ 대부분이 빚더미 위에 올라서 돈많은 척하는거지 솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음 지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는\nSC제일은행 · l********\n조회수\n15K\n좋아요\n201\n댓글\n145\n작성시간\n4일\n북마크'

In [147]:
a.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').text

'다른건 몰라도 돈 많은 사람이 많다는 말은 개구라야ㅎㅎ'

In [148]:
a.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').get_attribute('href')

'https://www.teamblind.com/kr/post/%EB%8B%A4%EB%A5%B8%EA%B1%B4-%EB%AA%B0%EB%9D%BC%EB%8F%84-%EB%8F%88-%EB%A7%8E%EC%9D%80-%EC%82%AC%EB%9E%8C%EC%9D%B4-%EB%A7%8E%EB%8B%A4%EB%8A%94-%EB%A7%90%EC%9D%80-%EA%B0%9C%EA%B5%AC%EB%9D%BC%EC%95%BC%E3%85%8E%E3%85%8E-26nihsck'

In [149]:
a.find_element(By.CLASS_NAME, "tit").find_element(By.CLASS_NAME, 'pre-txt').text

'내가 담당했던 업무 (지금은 아님)가 Kb부자리포트 같은 곳에 반영되는 고객들 금융자산 현황 만들었었거덩 당행뿐 아니라 타행들것도 같이 볼수있는데 거의 전국민의 금융자산 현황을 매일 보는 업무를 했었음ㅎㅎ 개인정보를 본건아니고 전체적인 현황 자료 그거보면서 느낀게 돈 많은 사람 겁나 많다는거 다 개구라 라는걸 느꼈다ㅎㅎ 대부분이 빚더미 위에 올라서 돈많은 척하는거지 솔직히 금융자산 자체가 비슷한 경제규모의 타국에 비해 현저히 적음 지들끼리는 나 빼고 다 돈 많아~ 우리나라에 돈많은 사람들이 얼마나 많은데 이러고있는'

In [150]:
a.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'name').text

'SC제일은행 · l********'

In [260]:
# Get Next Sibling Object from div
a=a.find_element(By.XPATH, './following-sibling::div')
print(a.get_attribute('class'))
a.text

article-list-pre nth-even


'현금부자 많다는거 다 구라엿네 ;;\nhttps://n.news.naver.com/article/025/0003452258?sid=101 6억이상 받아서 가격 받혔던거라 조정 꽤 나올듯... 아니 현금부자들 많아서 다 대출 적게받았다면서 형들 이게 무슨일이야\n삼성바이오로직스 · i*********\n조회수\n947\n좋아요\n4\n댓글\n7\n작성시간\n6시간\n북마크'

In [159]:
a.get_attribute('class').endswith('ad')

True

In [254]:
# If None of next sibling from article-list, scrolling on the DOM body and repeat(recursive) to get sibling object
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)    

In [7]:
def get_article_list(url, crawl_cnt) :
    driver.get(url)
    driver.implicitly_wait(3)
    time.sleep(2)  # 페이지 로딩 대기

    article_pre_data = []

    # 게시물 첫번째거 설정(최초 1회, 이후에는 sibling 으로 설정)
    article_preview=driver.find_element(By.CLASS_NAME, "article-list-pre")

    for i in range(0, crawl_cnt+1) :
        article_object={}
        
        # 광고로 게시물인 경우 넘어간다
        # 게시물 text 가 'NOW\n' or 'HOT\n' 으로 시작하면 추천게시물이므로 넘어간다
        # 이외의 경우 반복 게시물 크롤링 시작
        if(article_preview.get_attribute('class').lower().endswith('ad') == False and
            article_preview.text.startswith('HOT\n') == False and
            article_preview.text.startswith('NOW\n') == False) :
            #title on list
            title = article_preview.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').text
            
            # article link
            link = article_preview.find_element(By.CLASS_NAME, "tit").find_element(By.XPATH, 'h3/a').get_attribute('href')
            
            # article pre-text
            pre_text = article_preview.find_element(By.CLASS_NAME, "tit").find_element(By.CLASS_NAME, 'pre-txt').text
            
            # article firm
            firm = article_preview.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'name').text.split('·')[0]
            
            # article id
            id = article_preview.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'name').text.split('·')[1]
        
            # article date
            date_before = article_preview.find_element(By.CLASS_NAME, "sub").find_element(By.CLASS_NAME, 'info_fnc').find_element(By.CLASS_NAME, "past").text.split('작성시간\n')[1]
            date=convert_date(date_before)
        
            article_object['title']=title
            article_object['link']=link
            article_object['pre_text']=pre_text
            article_object['firm']=firm
            article_object['id']=id
            article_object['date']=date
            
            print(f"[{i}번째] {article_object}")
            
            article_pre_data.append(article_object)
            
        try :
            article_preview = article_preview.find_element(By.XPATH, './following-sibling::div')
        # 스크롤링의 끝인 경우 새로운 스크롤링 수행 후 계속 크로링 수행
        except NoSuchElementException as e:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)    
            
            print("### 스크롤링 완료###")
        
    return article_pre_data

In [8]:
list = get_article_list(site_url, 50000)

[2번째] {'title': '분양권 계약시 모델하우스에서 뭐 챙겨주나요?', 'link': 'https://www.teamblind.com/kr/post/%EB%B6%84%EC%96%91%EA%B6%8C-%EA%B3%84%EC%95%BD%EC%8B%9C-%EB%AA%A8%EB%8D%B8%ED%95%98%EC%9A%B0%EC%8A%A4%EC%97%90%EC%84%9C-%EB%AD%90-%EC%B1%99%EA%B2%A8%EC%A3%BC%EB%82%98%EC%9A%94-r3qds1xv', 'pre_text': '분양자들 방에 들어가보니 현금 지원 받으신 분들 많던데 맞나요?? (시행사에서 전체 다 주는 페이백 말구요)', 'firm': '한국공항공사 ', 'id': ' l*********', 'date': '2025-07-02'}
[3번째] {'title': '부동산 안정화 방법은 누가 뭐래도', 'link': 'https://www.teamblind.com/kr/post/%EB%B6%80%EB%8F%99%EC%82%B0-%EC%95%88%EC%A0%95%ED%99%94-%EB%B0%A9%EB%B2%95%EC%9D%80-%EB%88%84%EA%B0%80-%EB%AD%90%EB%9E%98%EB%8F%84-ictijlvx', 'pre_text': '이명박이 최고지. 정책이고 뭐고 무조건 집지어서 공급 늘리는게 최고임. 최악은 문재인. 공급없이 벌집 이곳 저곳 다 쑤셔서 전국 폭등 시킴.', 'firm': '현대자동차 ', 'id': ' i*******', 'date': '2025-07-02'}
[5번째] {'title': '자양동 어때', 'link': 'https://www.teamblind.com/kr/post/%EC%9E%90%EC%96%91%EB%8F%99-%EC%96%B4%EB%95%8C-0rr55rvo', 'pre_text': '아파트들 몰려있고 단지들 사이에 학교있던데 좋아보였거든? 연옌도 산다고해서 보게됐어 근데 동네자체가 중국인 동네인가?', 'fir

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.49)
Stacktrace:
0   chromedriver                        0x0000000101348ea4 cxxbridge1$str$ptr + 2722840
1   chromedriver                        0x0000000101340dac cxxbridge1$str$ptr + 2689824
2   chromedriver                        0x0000000100e923ec cxxbridge1$string$len + 90648
3   chromedriver                        0x0000000100e7bdcc chromedriver + 196044
4   chromedriver                        0x0000000100e9eef4 cxxbridge1$string$len + 142624
5   chromedriver                        0x0000000100f017d8 cxxbridge1$string$len + 546308
6   chromedriver                        0x0000000100f1a454 cxxbridge1$string$len + 647808
7   chromedriver                        0x0000000100ecd834 cxxbridge1$string$len + 333408
8   chromedriver                        0x000000010130bfc0 cxxbridge1$str$ptr + 24732

In [ ]:
df = pd.DataFrame(list)
df.to_csv("blind_topic_realestate.csv","utf-8")
df

,title,link,pre_text,firm,id,date
0,올포파는 2년 살아야 하나요?,https://www.teamblind.com/kr/post/%EC%98%AC%ED...,제곧내 입니다 바로 못 파나요?,ex-프레시지,꾸**,2025-07-02
1,폭락은 안바람 그냥 오르지만 않으면 됌,https://www.teamblind.com/kr/post/%ED%8F%AD%EB...,물가상승률 1-2프로정도씩만 오르면 된다 근데 아이러니하게도 그렇게되면 가격이 떨어...,한국과학기술연구원,사******,2025-07-02
2,방3개 + 알파룸 vs 방3개 + 펜트리\n투표,https://www.teamblind.com/kr/post/%EB%B0%A93%E...,요새 애기 3명있는집 드물거고 알파룸 트면 주방 확 트여서 들어오자마자 와 넓다 느...,NH농협,i*******,2025-07-02
3,전세보증금 반환대출 규제가 치명적인듯,https://www.teamblind.com/kr/post/%EC%A0%84%EC...,이전에는 전세낀 매물로 이리저리 옮겨다니며 덩치불리기가 가능했는데 이제는 한번 물리...,삼성전자,q********,2025-07-02
4,지금 정부는 집값을 떨어뜨릴려고하는게 아님,https://www.teamblind.com/kr/post/%EC%A7%80%EA...,집값이 오르지만 않아도 사람들이 집을 살려고 하지않음 오르지도 않을껄 뭐하러 빚내서...,KT,i*********,2025-07-02
5,중개수수료 1500만원인데 깎는 비결 좀 있을까요,https://www.teamblind.com/kr/post/%EC%A4%91%EA...,상한요율 0.7%해서 약 1500만원인데 너무 크네요 잘깎는 비결 좀 있을까요 ㅜㅜ,KB캐피탈,n*****,2025-07-02
6,공시지가 구간별 누진 보유세 가자!! 청원해야지,https://www.teamblind.com/kr/post/%EA%B3%B5%EC...,공시지가 구간별로 차등 세율 적용한 보유세 민주당 싱크탱크에 청원 넣어야지 ㅋㅋㅋ ...,공무원,i*******,2025-07-02


In [379]:
df['article_doc'] = df.apply(lambda x : get_article(x.link), axis=1)

[세부 글] {'title': '올포파는 2년 살아야 하나요?', 'body': '제곧내 입니다\n바로 못 파나요?', 'firm': 'ex-프레시지', 'id': '· 꾸**  ', 'like_cnt': '1', 'comment_cnt': '2', 'comment_map': [{'firm': '연세대학교의료원\n', 'id': ' k*****', 'contents': '\n실거주의무있습니다\n'}, {'firm': '삼성SDS\n', 'id': ' |********', 'contents': '\n올파포야\n'}]}
[세부 글] {'title': '방3개 + 알파룸 vs 방3개 + 펜트리', 'body': '요새 애기 3명있는집 드물거고\n\n알파룸 트면 주방 확 트여서 들어오자마자 와 넓다 느낌 날거같은데\n\n1. 알파룸 유지 - 애기방 , 컴퓨터방 , 옷방 등 사용\n\n2. 알파룸 트기 - 주방 확장으로 거실이 확 트임 , 펜트리 설치로 수납 기능\n\n뭐가 전세 더 잘 나갈까??\n\n2번은 +700만원 옵션(확장 및 주방 특화)', 'firm': 'NH농협', 'id': '· i*******  ', 'like_cnt': '1', 'comment_cnt': '1', 'comment_map': [{'firm': '한일시멘트\n', 'id': ' n*****', 'contents': '\n전세면 돈 들인만큼 큰 차이는 없으니\n돈 안쓰는게 나을듯!\n'}]}
[세부 글] {'title': '전세보증금 반환대출 규제가 치명적인듯', 'body': '이전에는 전세낀 매물로 이리저리 옮겨다니며 덩치불리기가 가능했는데\n이제는 한번 물리면 답이 없다', 'firm': '삼성전자', 'id': '· q********  ', 'like_cnt': '3', 'comment_cnt': '2', 'comment_map': [{'firm': '한국애보트\n', 'id': ' ㅇ********', 'contents': '\n그치 ㅠ\n'}, {'firm': '크래프톤\n',

In [377]:
df.loc[6]

title                              공시지가 구간별 누진 보유세 가자!! 청원해야지
link        https://www.teamblind.com/kr/post/%EA%B3%B5%EC...
pre_text    공시지가 구간별로 차등 세율 적용한 보유세 민주당 싱크탱크에 청원 넣어야지 ㅋㅋㅋ ...
firm                                                     공무원 
id                                                   i*******
date                                               2025-07-02
Name: 6, dtype: object